In [9]:
# Importation des libraries et packages que nous allons utiliser
import pandas as pd
import numpy as np
import spacy
import random
from spacy.training.example import Example
from itertools import product
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# I) Preprcessing

## I) 1- Importation des donnees

In [11]:
# Ouvrir les fichiers texte (Train.txt, dev.txt et Test.txt) et lire son contenu ligne par ligne
with open("../train.txt", "r", encoding="utf-8") as file:
    train = file.readlines()
    
with open("../test.txt", "r", encoding="utf-8") as file:
    test = file.readlines()

## I) 2- Definition du model

In [12]:
# Definition du model sous forme de classe
class NER_WOLOF:
        
    def __init__(self):
        
        pass
    
    '''Definition d'une methode qui formate les donnees texte au format que consomme Spacy cad une liste ou chaque
    represente un tuple dont le 1er element est une phrase et le 2em un dictionnaire avec comme clé entitie et comme
    valeur une liste des entitées nommees qui sont des tuples de 3 elements : les index de debut et fin et les entites
    '''
    

    def format_train(self, X):
        # Liste de lignes au format CoNLL
        self.X = X
        # Initialisation de la liste TRAIN_DATA
        data = []

        # Initialisation des variables pour collecter les phrases et leurs annotations
        phrase = []
        annotations = []

        # Parcourir les lignes
        for ligne in self.X:
            # Ignorez les lignes vides
            if not ligne.strip():
                # Si c'est une ligne vide, cela signifie la fin de la phrase précédente
                # Ajoutez la phrase précédente avec ses annotations à TRAIN_DATA
                if phrase and annotations:
                    data.append((" ".join(phrase), {'entities': annotations}))

                # Réinitialisez les variables pour la nouvelle phrase
                phrase = []
                annotations = []
            else:
                # Séparez chaque ligne en tokens et étiquettes
                token, entite = ligne.split()

                # regrouper les entites nommees
                if entite in ["B-PER", "I-PER"]:
                    entite = "PER"
                if entite in ["B-ORG", "I-ORG"]:
                    entite = "ORG"
                if entite in ["B-DATE", "I-DATE"]:
                    entite = "DATE"
                if entite in ["B-LOC", "I-LOC"]:
                    entite = "LOC"


                # Ajoutez le token à la phrase
                phrase.append(token)

                # Si c'est une entité, ajoutez son début, sa fin et son type aux annotations
                if entite != 'O':
                    annotations.append((len(' '.join(phrase)) - len(token), len(' '.join(phrase)), entite))
                    
        return data
                    
    
    def format_test (self, X):
            
        self.X = X
        # Initialisation de la liste data
        data = []

        # Initialisation des variables pour collecter les phrases et leurs annotations
        phrase = []
        annotations = []

        # Parcourir les lignes
        for ligne in self.X:
            # Ignorez les lignes vides
            if not ligne.strip():
                # Si c'est une ligne vide, cela signifie la fin de la phrase précédente
                # Ajoutez la phrase précédente avec ses annotations à TRAIN_DATA
                if phrase and annotations:
                    data.append((" ".join(phrase), {'entities': annotations}))

                # Réinitialisez les variables pour la nouvelle phrase
                phrase = []
                annotations = []
            else:
                
                # Séparez chaque ligne en tokens et étiquettes
                token, entite = ligne.split()
                
                # regrouper les entites nommees
                if entite in ["B-PER", "I-PER"]:
                    entite = "PER"
                if entite in ["B-ORG", "I-ORG"]:
                    entite = "ORG"
                if entite in ["B-DATE", "I-DATE"]:
                    entite = "DATE"
                if entite in ["B-LOC", "I-LOC"]:
                    entite = "LOC"

                # Ajoutez le token à la phrase
                phrase.append(token)

                # ajoutez son début, sa fin et l'entité nommée aux annotations
                annotations.append((len(' '.join(phrase)) - len(token), len(' '.join(phrase)), entite))
                    
        return data
    
    
    
    ''' Definition d'une fonction qui instancie un model vide de Spacy et qui l'entraine selon nos donnees etiquettees
    et les hyperparametres qu'on lui aura donnes
    
    '''
    
    def fit(self, train, n_iter, size, drop):
        
        self.train = train
        self.n_iter = n_iter
        self.size = size
        self.drop = drop
        
        # Création d'un modèle vide
        nlp = spacy.blank("xx")

        # Creation d'un composant NER
        ner = nlp.add_pipe("ner")

        # Ajout des étiquettes d'entités personnalisées
        ner.add_label("PER")
        ner.add_label("ORG")
        ner.add_label("LOC")
        ner.add_label("DATE")


        # Entrainement du modèle avec l'optimizer de base de Spacy Adam qui est plus adapté au nlp
        optimizer = nlp.begin_training()
        # stockage des valeurs de perte au fil des iterations
        loss_values = []  

        for itn in range(self.n_iter):
            #random.shuffle(self.train)
            losses = {}

            # Divisez les données en minibatch
            for batch in spacy.util.minibatch(self.train, size = self.size):
                texts, annotations = zip(*batch)
                example = []
                for i in range(len(texts)):
                    doc = nlp.make_doc(texts[i])
                    example.append(Example.from_dict(doc, annotations[i]))

                # Mettez à jour le modèle
                nlp.update(example, drop = self.drop, losses = losses)

            loss_values.append(losses["ner"])
            print(losses, " Iter n: ", itn)

        return nlp

    ''' Definition d'une methode qui permet de predire un jeu de données '''
    
    # Cette methode permet d'éclater les PER en B-PER, I-PER et pour les autres étiquettes
    '''On itere sur la liste de phrases et on va recuperer les étiquettes pour iterer sur les etiquettes de la phrase
    pour la 1ere iteration on ajoute le préfixe "B-" sinon on verifie si l'entite précédente est la meme et qu'elles 
    se suivent si c'est le cas on ajoute le prefixe "I-" sinon dans tous les autres cas on ajoute le prefixe "B-"
    '''
    def _predict (self, X) :
        self.X = X
        L = []
        for x in self.X:
            l = []
            for j in range(len(x)) :
                if x[j][2] == "O":
                    l.append((x[j][0], x[j][1], "O"))
                elif j == 0 and  x[j][2] != "O":
                    l.append((x[j][0], x[j][1], "B-" + x[j][2]))
                elif (x[j-1][2] == x[j][2] or x[j-1][2] == "I-" + x[j][2]) and x[j-1][1] == x[j][0] - 1 :
                    l.append((x[j][0], x[j][1], "I-" + x[j][2]))
                else :
                    l.append((x[j][0], x[j][1], "B-" + x[j][2]))

            L.extend(l)

        return L


        return 
    
    # Methode pour prédire des donnees de format Spacy
    def predict(self, model, test):
        
        self.nlp = model
        self.test = test
        
        # Analysez les données de test avec le modèle
        predictions = []
        annotations = []

        for texte, annotation_correcte in self.test:
            # on recupere les etiquette de la phrase qu'on ajoute à la liste annotations
            annotations.append(annotation_correcte['entities'])
            # on fait de meme avec une nouvelle liste de prediction ponctuelle
            pred = annotation_correcte['entities']
            pred = [(x[0], x[1], "O") for x in pred]
            # on itere sur la liste des predictions ponctuelle avec que des etiquettes a "0"
            for i in range(len(pred)):
                # Prediction avec le model
                if self.nlp(texte[pred[i][0] : pred[i][1]]).ents :
                    pred[i] = (pred[i][0], pred[i][1], self.nlp(texte[pred[i][0] : pred[i][1]]).ents[0].label_ )


            # on ajoute la prediction ponctuelle à la liste de prediction
            predictions.append(pred)
        
        annotations = self._predict(annotations)
        predictions = self._predict(predictions)
        
        # recuperation des labels pour le rapport
        annotations = [annotation[2] for annotation in annotations]
        predictions = [prediction[2] for prediction in predictions]

        return annotations, predictions
    
    # Methode pour predire des Phrases
    def predict_phrase (self, model, X) :
        
        self.model = model
        self.X = X

        L = []
        for x in self.X.split() :
            if self.model(x).ents :
                L.append(self.model(x).ents[0].label_)
            else :
                L.append("O")
        
        return L
            
    
    ''' Implementation d'une methode score'''
    
    def score (self, y_true, y_pred):
        self.annotations = y_true
        self.predictions = y_pred
        # Calculez les scores de performance
        report = classification_report(self.annotations, self.predictions)
        
        # Recuperation du weigthed avg qui donne le score en prennant compte des dimension disproportionnees de chaque classe
        score = float(report.split()[-8])
        
        return score


## I) 3- Formattage des donnees

In [13]:
# Instanciation de la classe NER_WOLOF
ner = NER_WOLOF()
# Utilisation de sa methode format_data pour formatter les donnees train, val et test
train_data = ner.format_train(train)
test_data = ner.format_test(test)

In [14]:
train_data

[('SAFIYETU BÉEY Céy Koronaa !',
  {'entities': [(0, 8, 'PER'), (9, 13, 'PER')]}),
 ('SÉEX TIIJAAN JÉEY ( Xeltukat / Enda - Cacid )',
  {'entities': [(0, 4, 'PER'),
    (5, 12, 'PER'),
    (13, 17, 'PER'),
    (31, 35, 'ORG'),
    (38, 43, 'ORG')]}),
 ('Ci tekkib Paap Aali Jàllo',
  {'entities': [(10, 14, 'PER'), (15, 19, 'PER'), (20, 25, 'PER')]}),
 ('Waaye , li ëpp ci samay xalaat ak i seetlu , Afrig la ñeel , sunu Afrig ak ëllëgam .',
  {'entities': [(45, 50, 'LOC'), (66, 71, 'LOC')]}),
 ('Bu ko defee , sama xel mi day faral di wër réewi Afrig yi ba ñu daj , bunt bu dul buntu jinne mu fëgg fa , dugg ci biir kër gi , jàngat la fay xew , doxalinu njiit ya fa nekk ak taxawaayu askan wa .',
  {'entities': [(49, 54, 'LOC')]}),
 ('Ndaxte , liy màndargaal daanaka réewi Afrig yépp mooy mbir yii may limsi : ndóol , xiif ak mar , ay xarey biir - réew yu dul jeex , ger , njàng mu yàqu yaxeet , ñàkkub liggéey , añs .',
  {'entities': [(38, 43, 'LOC')]}),
 ('Waaye , dinaa leen baaxe , ci sama xa

In [15]:
model = ner.fit(train_data, 75, 32, 0.5)

{'ner': 8179.08870783742}  Iter n:  0
{'ner': 2776.9794977890106}  Iter n:  1
{'ner': 2072.2307997187845}  Iter n:  2
{'ner': 1778.65288297868}  Iter n:  3
{'ner': 1550.2671765010643}  Iter n:  4
{'ner': 1421.9011126027372}  Iter n:  5
{'ner': 1248.8723479423081}  Iter n:  6
{'ner': 1072.3792438477624}  Iter n:  7
{'ner': 929.3846038907363}  Iter n:  8
{'ner': 851.3473664674036}  Iter n:  9
{'ner': 772.9753691156399}  Iter n:  10
{'ner': 674.4358066215862}  Iter n:  11
{'ner': 642.2582403622405}  Iter n:  12
{'ner': 560.1120726906752}  Iter n:  13
{'ner': 549.4749393486775}  Iter n:  14
{'ner': 488.0868038406417}  Iter n:  15
{'ner': 430.9982072524866}  Iter n:  16
{'ner': 422.4623120954685}  Iter n:  17
{'ner': 396.1295364810721}  Iter n:  18
{'ner': 344.4586908203446}  Iter n:  19
{'ner': 310.38255518217943}  Iter n:  20
{'ner': 260.54838837873444}  Iter n:  21
{'ner': 301.6628026417836}  Iter n:  22
{'ner': 283.1952797663606}  Iter n:  23
{'ner': 264.35451482567083}  Iter n:  24
{'n

In [16]:
model = spacy.load("NER")

c:\Users\Don Arhona\Desktop\IA_Competition\venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'xx_pipeline' (0.0.0) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [17]:
y_true, y_pred = ner.predict(model, test_data)

In [18]:
# N_iter : 50, size : 16, drop : 0.4

# Calculez les scores de performance
report = classification_report(y_true, y_pred)
print(report)
# Recuperation du weigthed avg qui donne le score en prennant compte des dimension disproportionnees de chaque classe
score = float(report.split()[-2])


              precision    recall  f1-score   support

      B-DATE       0.28      0.36      0.31        70
       B-LOC       0.72      0.66      0.69       211
       B-ORG       0.24      0.31      0.27        55
       B-PER       0.31      0.44      0.37       176
      I-DATE       1.00      0.05      0.09       153
       I-LOC       0.00      0.00      0.00         4
       I-ORG       0.00      0.00      0.00        13
       I-PER       0.88      0.30      0.44       193
           O       0.97      0.99      0.98     10807

    accuracy                           0.94     11682
   macro avg       0.49      0.34      0.35     11682
weighted avg       0.94      0.94      0.93     11682



In [21]:
N_iter : 75; size : 16; drop : 0.4

#Calculez les scores de performance
report = classification_report(y_true, y_pred)
print(report)
#Recuperation du weigthed avg qui donne le score en prennant compte des dimension disproportionnees de chaque classe
score = float(report.split()[-2])


              precision    recall  f1-score   support

      B-DATE       0.28      0.36      0.31        70
       B-LOC       0.72      0.66      0.69       211
       B-ORG       0.24      0.31      0.27        55
       B-PER       0.31      0.44      0.37       176
      I-DATE       1.00      0.05      0.09       153
       I-LOC       0.00      0.00      0.00         4
       I-ORG       0.00      0.00      0.00        13
       I-PER       0.88      0.30      0.44       193
           O       0.97      0.99      0.98     10807

    accuracy                           0.94     11682
   macro avg       0.49      0.34      0.35     11682
weighted avg       0.94      0.94      0.93     11682



In [22]:
N_iter : 50; size : 16; drop : 0.4

#Calculez les scores de performance
report = classification_report(y_true, y_pred)
print(report)
# Recuperation du weigthed avg qui donne le score en prennant compte des dimension disproportionnees de chaque classe
score = float(report.split()[-2])


              precision    recall  f1-score   support

      B-DATE       0.28      0.36      0.31        70
       B-LOC       0.72      0.66      0.69       211
       B-ORG       0.24      0.31      0.27        55
       B-PER       0.31      0.44      0.37       176
      I-DATE       1.00      0.05      0.09       153
       I-LOC       0.00      0.00      0.00         4
       I-ORG       0.00      0.00      0.00        13
       I-PER       0.88      0.30      0.44       193
           O       0.97      0.99      0.98     10807

    accuracy                           0.94     11682
   macro avg       0.49      0.34      0.35     11682
weighted avg       0.94      0.94      0.93     11682



In [23]:
N_iter : 150; size : 16; drop : 0.4

# Calculez les scores de performance
report = classification_report(y_true, y_pred)
print(report)
# Recuperation du weigthed avg qui donne le score en prennant compte des dimension disproportionnees de chaque classe
score = float(report.split()[-2])


              precision    recall  f1-score   support

      B-DATE       0.28      0.36      0.31        70
       B-LOC       0.72      0.66      0.69       211
       B-ORG       0.24      0.31      0.27        55
       B-PER       0.31      0.44      0.37       176
      I-DATE       1.00      0.05      0.09       153
       I-LOC       0.00      0.00      0.00         4
       I-ORG       0.00      0.00      0.00        13
       I-PER       0.88      0.30      0.44       193
           O       0.97      0.99      0.98     10807

    accuracy                           0.94     11682
   macro avg       0.49      0.34      0.35     11682
weighted avg       0.94      0.94      0.93     11682



In [25]:
N_iter : 100; size : 32; drop : 0.5

# Calculez les scores de performance
report = classification_report(y_true, y_pred)
print(report)
# Recuperation du weigthed avg qui donne le score en prennant compte des dimension disproportionnees de chaque classe
score = float(report.split()[-2])


              precision    recall  f1-score   support

      B-DATE       0.28      0.36      0.31        70
       B-LOC       0.72      0.66      0.69       211
       B-ORG       0.24      0.31      0.27        55
       B-PER       0.31      0.44      0.37       176
      I-DATE       1.00      0.05      0.09       153
       I-LOC       0.00      0.00      0.00         4
       I-ORG       0.00      0.00      0.00        13
       I-PER       0.88      0.30      0.44       193
           O       0.97      0.99      0.98     10807

    accuracy                           0.94     11682
   macro avg       0.49      0.34      0.35     11682
weighted avg       0.94      0.94      0.93     11682



In [26]:
import spacy

# Charger le modèle SpaCy (remplacez le nom du modèle par le vôtre)
ner_model = spacy.load("NER")

# Sauvegarder le modèle dans un répertoire
ner_model.to_disk(r"C:\Users\Don Arhona\Desktop\IA_Competition\NerApp\ModelNER")


c:\Users\Don Arhona\Desktop\IA_Competition\venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'xx_pipeline' (0.0.0) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
